In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 10, 4
plt.style.use('ggplot')

# Seleccion de estrategias y generacion del portfolio

In [2]:
# only colab environment
#from google.colab import drive  
#drive.mount('/content/drive')
# v_path = '/content/drive/MyDrive/RL_DIC/BENCHMARK/' 

v_path = '' # local environment

In [3]:
# definimos los activos que queremos en el portfolio: NQ, ES, YM, CL, NG, GC, VX, TY
markets = ['NQ', 'ES', 'YM', 'CL', 'NG', 'GC', 'VX', 'TY']

In [4]:
# leo los ficheros de las estrategias preseleccionadas para cada activo en los baacktests long y short
df_nq_long_bkt = pd.read_pickle(v_path + 'trading_systems_NQ/NQ_ratios__2018-09-01--2022-09-01.pkl')
df_nq_short_bkt = pd.read_pickle(v_path + 'trading_systems_NQ/NQ_ratios__2022-09-01--2023-01-01.pkl')
df_es_long_bkt = pd.read_pickle(v_path + 'trading_systems_ES/ES_ratios__2018-09-01--2022-09-01.pkl')
df_es_short_bkt = pd.read_pickle(v_path + 'trading_systems_ES/ES_ratios__2022-09-01--2023-01-01.pkl')
df_ym_long_bkt = pd.read_pickle(v_path + 'trading_systems_YM/YM_ratios__2018-09-01--2022-09-01.pkl')
df_ym_short_bkt = pd.read_pickle(v_path + 'trading_systems_YM/YM_ratios__2022-09-01--2023-01-01.pkl')
df_cl_long_bkt = pd.read_pickle(v_path + 'trading_systems_CL/CL_ratios__2018-09-01--2022-09-01.pkl')
df_cl_short_bkt = pd.read_pickle(v_path + 'trading_systems_CL/CL_ratios__2022-09-01--2023-01-01.pkl')
df_ng_long_bkt = pd.read_pickle(v_path + 'trading_systems_NG/NG_ratios__2018-09-01--2022-09-01.pkl')
df_ng_short_bkt = pd.read_pickle(v_path + 'trading_systems_NG/NG_ratios__2022-09-01--2023-01-01.pkl')
df_gc_long_bkt = pd.read_pickle(v_path + 'trading_systems_GC/GC_ratios__2018-09-01--2022-09-01.pkl')
df_gc_short_bkt = pd.read_pickle(v_path + 'trading_systems_GC/GC_ratios__2022-09-01--2023-01-01.pkl')
#df_vx_long_bkt = pd.read_pickle(v_path + 'trading_systems_VX/VX_ratios__2018-09-01--2022-09-01.pkl')
#df_vx_short_bkt = pd.read_pickle(v_path + 'trading_systems_VX/VX_ratios__2022-09-01--2023-01-01.pkl')
#df_ty_long_bkt = pd.read_pickle(v_path + 'trading_systems_TY/TY_ratios__2018-09-01--2022-09-01.pkl')
#df_ty_short_bkt = pd.read_pickle(v_path + 'trading_systems_TY/TY_ratios__2022-09-01--2023-01-01.pkl')


In [5]:
# definimos una funcion que identifica las estrategias comunes a ambos backtests
def intersection(df_long, df_short):
    df_long['id'] = df_long['market'] + df_long['timeframe'] + df_long['system_type'] + df_long['in_condition_1'] + df_long['in_condition_2'] + df_long['out_condition_1'] + df_long['out_condition_2']
    df_short['id'] = df_short['market'] + df_short['timeframe'] + df_short['system_type'] + df_short['in_condition_1'] + df_short['in_condition_2'] + df_short['out_condition_1'] + df_short['out_condition_2']
    df_long['id'] = df_long['id'].astype(str)
    df_short['id'] = df_short['id'].astype(str)
    df_long = df_long.set_index('id')
    df_short = df_short.set_index('id')
    df_intersection = df_long.join(df_short, lsuffix='_long', rsuffix='_short')
    df_intersection = df_intersection.dropna()
    df_intersection = df_intersection.reset_index()
    df_intersection = df_intersection.drop(['id', 'market_short', 'timeframe_short', 'system_type_short', 'in_condition_1_short', 'in_condition_2_short', 'out_condition_1_short', 'out_condition_2_short'], axis=1)
    df_intersection.columns = ['market', 'timeframe', 'system_type', 'in_condition_1', 'in_condition_2', 'out_condition_1', 'out_condition_2', 'n_trades_long_bkt', 'profit_factor_long_bkt', 'bmo_long_bkt', 'net_profit_long_bkt', 'n_trades_short_bkt', 'profit_factor_short_bkt', 'bmo_short_bkt', 'net_profit_short_bkt']

    return df_intersection

In [6]:
# definimos una funcion que obtenga la mejor estrategia de cada grupo timeframe-system_type
def best_strategy(df_intersection):

    # para cada grupo de diferente tmeframe y system_type, podemos seleccionar la estrategia que tenga mayor NP, BMO o PF. Elejimos la que tenga mayor NP
    df_intersection = df_intersection.sort_values(['timeframe', 'system_type', 'net_profit_long_bkt', 'net_profit_short_bkt'], ascending=[True, True, False, False]) # mejor net profit
    #df_intersection = df_intersection.sort_values(['timeframe', 'system_type', 'bmo_long_bkt', 'bmo_short_bkt'], ascending=[True, True, False, False]) # mejor bmo 
    #df_intersection = df_intersection.sort_values(['timeframe', 'system_type', 'profit_factor_long_bkt', 'profit_factor_short_bkt'], ascending=[True, True, False, False]) # mejor profit factor
    df_intersection = df_intersection.drop_duplicates(['timeframe', 'system_type'])

    return df_intersection

In [7]:
# para cada mercado obtenemos la intersection y best strategy, y lo vamos almacenanado en df_portfolio
df_portfolio = pd.DataFrame()
df_portfolio = df_portfolio.append(best_strategy(intersection(df_nq_long_bkt, df_nq_short_bkt)))
df_portfolio = df_portfolio.append(best_strategy(intersection(df_es_long_bkt, df_es_short_bkt)))
df_portfolio = df_portfolio.append(best_strategy(intersection(df_ym_long_bkt, df_ym_short_bkt)))
df_portfolio = df_portfolio.append(best_strategy(intersection(df_cl_long_bkt, df_cl_short_bkt)))
df_portfolio = df_portfolio.append(best_strategy(intersection(df_ng_long_bkt, df_ng_short_bkt)))
df_portfolio = df_portfolio.append(best_strategy(intersection(df_gc_long_bkt, df_gc_short_bkt)))
#df_portfolio = df_portfolio.append(best_strategy(intersection(df_ty_long_bkt, df_ty_short_bkt)))
#df_portfolio = df_portfolio.append(best_strategy(intersection(df_vx_long_bkt, df_vx_short_bkt)))
df_portfolio = df_portfolio.reset_index()
df_portfolio = df_portfolio.drop(['index'], axis=1)

In [8]:
# lo guardamos en un fichero csv
df_portfolio.to_csv('df_portfolio_conditions.csv')
df_portfolio

,market,timeframe,system_type,in_condition_1,in_condition_2,out_condition_1,out_condition_2,n_trades_long_bkt,profit_factor_long_bkt,bmo_long_bkt,net_profit_long_bkt,n_trades_short_bkt,profit_factor_short_bkt,bmo_short_bkt,net_profit_short_bkt
0,NQ,15min,trend_short,b_condition_2,b_condition_24,a_condition_12,a_condition_18,573,1.418019,188.830716,108200.0,51,1.698507,390.000000,19890.0
1,NQ,5min,trend_short,b_condition_1,b_condition_23,a_condition_13,a_condition_25,559,1.482666,148.193202,82840.0,50,1.897814,365.500000,18275.0
2,ES,10min,trend_long,a_condition_14,a_condition_26,b_condition_22,b_condition_24,504,1.412893,135.664683,68375.0,49,1.834465,407.653061,19975.0
3,ES,15min,trend_long,a_condition_2,a_condition_25,b_condition_12,b_condition_22,501,1.545865,141.641717,70962.5,59,1.695652,271.186441,16000.0
4,YM,10min,trend_long,a_condition_11,a_condition_27,b_condition_12,b_condition_20,587,1.411334,91.873935,53930.0,62,1.626443,183.790323,11395.0
5,YM,15min,trend_long,a_condition_2,a_condition_25,b_condition_12,b_condition_22,546,1.472367,100.109890,54660.0,55,1.919438,243.818182,13410.0
6,YM,15min,trend_short,b_condition_8,b_condition_24,a_condition_10,a_condition_17,595,1.407323,87.126050,51840.0,49,1.622726,171.122449,8385.0
7,NG,10min,trend_long,a_condition_1,a_condition_27,b_condition_6,b_condition_11,508,1.496451,117.027559,59450.0,41,1.820547,453.902439,18610.0
8,NG,10min,trend_short,b_condition_10,b_condition_26,a_condition_4,a_condition_20,813,1.428588,89.876999,73070.0,71,2.370724,586.901408,41670.0
9,NG,15min,trend_long,a_condition_2,a_condition_25,b_condition_10,b_condition_26,661,1.465162,100.090772,66160.0,49,2.416885,551.428571,27020.0


In [9]:
# creamos la leyenda de condiciones que se usaron en la generacion de estrategias

df_portfolio = df_portfolio.replace(to_replace='a_condition_1', value='3_bars_up')
df_portfolio = df_portfolio.replace(to_replace='a_condition_2', value='2_bars_up')
df_portfolio = df_portfolio.replace(to_replace='a_condition_3', value='close > bb_upband_20__2')
df_portfolio = df_portfolio.replace(to_replace='a_condition_4', value='close > max_6')
df_portfolio = df_portfolio.replace(to_replace='a_condition_5', value='close > max_14')
df_portfolio = df_portfolio.replace(to_replace='a_condition_6', value='close > max_200')
df_portfolio = df_portfolio.replace(to_replace='a_condition_7', value='close > ema_6')
df_portfolio = df_portfolio.replace(to_replace='a_condition_8', value='close > ema_12')
df_portfolio = df_portfolio.replace(to_replace='a_condition_9', value='close > ema_26')
df_portfolio = df_portfolio.replace(to_replace='a_condition_10', value='close > sma_60')
df_portfolio = df_portfolio.replace(to_replace='a_condition_11', value='close > sma_120')
df_portfolio = df_portfolio.replace(to_replace='a_condition_12', value='close > sma_200')
df_portfolio = df_portfolio.replace(to_replace='a_condition_13', value='close > (close.shift(2) + 1.4 * atr_14)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_14', value='close > (close.shift(2) + 2 * atr_14)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_15', value='close > (close.shift(2) + 1.4 * atr_23)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_16', value='close > (close.shift(2) + 2 * atr_23)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_17', value='close > (close.shift(2) + 1.4 * atr_92)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_18', value='close > (close.shift(2) + 2 * atr_92)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_19', value='close > (close.shift(2) + 1.4 * atr_138)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_20', value='close > (close.shift(2) + 2 * atr_138)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_21', value='close > (close.shift(2) + 1.4 * atr_276)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_22', value='close > (close.shift(2) + 2 * atr_276)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_23', value='close > (close.shift(2) + 2.8 * atr_14)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_24', value='close > (close.shift(2) + 2.8 * atr_23)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_25', value='close > (close.shift(2) + 2.8 * atr_92)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_26', value='close > (close.shift(2) + 2.8 * atr_138)')
df_portfolio = df_portfolio.replace(to_replace='a_condition_27', value='close > (close.shift(2) + 2.8 * atr_276)')

df_portfolio = df_portfolio.replace(to_replace='b_condition_1', value='3_bars_down')
df_portfolio = df_portfolio.replace(to_replace='b_condition_2', value='2_bars_down')
df_portfolio = df_portfolio.replace(to_replace='b_condition_3', value='close < bb_downband_20__2')
df_portfolio = df_portfolio.replace(to_replace='b_condition_4', value='close < min_6')
df_portfolio = df_portfolio.replace(to_replace='b_condition_5', value='close < min_14')
df_portfolio = df_portfolio.replace(to_replace='b_condition_6', value='close < min_200')
df_portfolio = df_portfolio.replace(to_replace='b_condition_7', value='close < ema_6')
df_portfolio = df_portfolio.replace(to_replace='b_condition_8', value='close < ema_12')
df_portfolio = df_portfolio.replace(to_replace='b_condition_9', value='close < ema_26')
df_portfolio = df_portfolio.replace(to_replace='b_condition_10', value='close < sma_60')
df_portfolio = df_portfolio.replace(to_replace='b_condition_11', value='close < sma_120')
df_portfolio = df_portfolio.replace(to_replace='b_condition_12', value='close < sma_200')
df_portfolio = df_portfolio.replace(to_replace='b_condition_13', value='close < (close.shift(2) - 1.4 * atr_14)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_14', value='close < (close.shift(2) - 2 * atr_14)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_15', value='close < (close.shift(2) - 1.4 * atr_23)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_16', value='close < (close.shift(2) - 2 * atr_23)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_17', value='close < (close.shift(2) - 1.4 * atr_92)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_18', value='close < (close.shift(2) - 2 * atr_92)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_19', value='close < (close.shift(2) - 1.4 * atr_138)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_20', value='close < (close.shift(2) - 2 * atr_138)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_21', value='close < (close.shift(2) - 1.4 * atr_276)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_22', value='close < (close.shift(2) - 2 * atr_276)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_23', value='close < (close.shift(2) - 2.8 * atr_14)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_24', value='close < (close.shift(2) - 2.8 * atr_23)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_25', value='close < (close.shift(2) - 2.8 * atr_92)')
df_portfolio = df_portfolio.replace(to_replace='b_condition_26', value='close < (close.shift(2) - 2.8 * atr_138)')  
df_portfolio = df_portfolio.replace(to_replace='b_condition_27', value='close < (close.shift(2) - 2.8 * atr_276)')


In [10]:
# lo guardamos en un fichero csv
df_portfolio.to_csv('df_portfolio_benchmark.csv')
df_portfolio

,market,timeframe,system_type,in_condition_1,in_condition_2,out_condition_1,out_condition_2,n_trades_long_bkt,profit_factor_long_bkt,bmo_long_bkt,net_profit_long_bkt,n_trades_short_bkt,profit_factor_short_bkt,bmo_short_bkt,net_profit_short_bkt
0,NQ,15min,trend_short,2_bars_down,close < (close.shift(2) - 2.8 * atr_23),close > sma_200,close > (close.shift(2) + 2 * atr_92),573,1.418019,188.830716,108200.0,51,1.698507,390.000000,19890.0
1,NQ,5min,trend_short,3_bars_down,close < (close.shift(2) - 2.8 * atr_14),close > (close.shift(2) + 1.4 * atr_14),close > (close.shift(2) + 2.8 * atr_92),559,1.482666,148.193202,82840.0,50,1.897814,365.500000,18275.0
2,ES,10min,trend_long,close > (close.shift(2) + 2 * atr_14),close > (close.shift(2) + 2.8 * atr_138),close < (close.shift(2) - 2 * atr_276),close < (close.shift(2) - 2.8 * atr_23),504,1.412893,135.664683,68375.0,49,1.834465,407.653061,19975.0
3,ES,15min,trend_long,2_bars_up,close > (close.shift(2) + 2.8 * atr_92),close < sma_200,close < (close.shift(2) - 2 * atr_276),501,1.545865,141.641717,70962.5,59,1.695652,271.186441,16000.0
4,YM,10min,trend_long,close > sma_120,close > (close.shift(2) + 2.8 * atr_276),close < sma_200,close < (close.shift(2) - 2 * atr_138),587,1.411334,91.873935,53930.0,62,1.626443,183.790323,11395.0
5,YM,15min,trend_long,2_bars_up,close > (close.shift(2) + 2.8 * atr_92),close < sma_200,close < (close.shift(2) - 2 * atr_276),546,1.472367,100.109890,54660.0,55,1.919438,243.818182,13410.0
6,YM,15min,trend_short,close < ema_12,close < (close.shift(2) - 2.8 * atr_23),close > sma_60,close > (close.shift(2) + 1.4 * atr_92),595,1.407323,87.126050,51840.0,49,1.622726,171.122449,8385.0
7,NG,10min,trend_long,3_bars_up,close > (close.shift(2) + 2.8 * atr_276),close < min_200,close < sma_120,508,1.496451,117.027559,59450.0,41,1.820547,453.902439,18610.0
8,NG,10min,trend_short,close < sma_60,close < (close.shift(2) - 2.8 * atr_138),close > max_6,close > (close.shift(2) + 2 * atr_138),813,1.428588,89.876999,73070.0,71,2.370724,586.901408,41670.0
9,NG,15min,trend_long,2_bars_up,close > (close.shift(2) + 2.8 * atr_92),close < sma_60,close < (close.shift(2) - 2.8 * atr_138),661,1.465162,100.090772,66160.0,49,2.416885,551.428571,27020.0
